# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyDo


In [3]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [5]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## And then enter Gradio's magic!

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>

In [6]:
import google.generativeai as genai

In [20]:
# Initialize Gemini (do this ONCE outside the chat function)
try:
    genai.configure(api_key=google_api_key)  # Replace with your actual API key
    model = genai.GenerativeModel('gemini-1.5-flash')
except Exception as e:
    print(f"Error initializing Gemini: {e}")
    model = None  # Set model to None to indicate failure

In [21]:
# Example prompt
system_prompt = "You are a freindly and helpful assistant. You are responsible to answer the questions to our client that \
visit our web site. You only responsible to answer the following types of product:\
1. printer\
2. web camera\
3. laptop\
if visitor ask question out of those scope, you can simplely answer, I dont know the answer.\
you need to provide introduce and suggestion to win purchase from vistor."

In [37]:
response = model.generate_content("Explain how AI works")
print(response.text)

Artificial intelligence (AI) is a broad field encompassing many techniques, but at its core, it aims to create systems that can perform tasks that typically require human intelligence.  There's no single "how it works" because different AI approaches use different methods.  However, we can break it down into key concepts:

**1. Data is King:**  AI systems learn from data. The more relevant and high-quality data they're trained on, the better they perform. This data can be anything from images and text to sensor readings and financial transactions.

**2. Algorithms are the Recipes:** Algorithms are sets of instructions that tell the AI system how to process and learn from the data.  Different algorithms are suited for different tasks.  Some common types include:

* **Machine Learning (ML):** This is a subfield of AI where systems learn from data without explicit programming.  Instead of being given rules, they identify patterns and relationships within the data to make predictions or de

In [35]:
def chat(message, history=[]):
    if model is None:
        yield "Gemini is not initialized. Please check your API key and network connection."
        return

    messages = [{"role": "system", "parts": [{"text": system_message}]}] + \
               [{"role": msg["role"], "parts": [{"text": msg["content"]}]} for msg in history] + \
               [{"role": "user", "parts": [{"text": message}]}]
    try:
        response = model.generate_content(messages)
        # Check for errors in the response
        # if response.prompt_feedback and response.prompt_feedback.block_reason:
        #     yield f"Response blocked due to: {response.prompt_feedback.block_reason}"
        #     return

        # Extract the text from the response
        if response.parts:
            text_response = "".join([part.text for part in response.parts])
            yield text_response
        else:
            yield "No text content in the response."

    except Exception as e:
        yield f"An error occurred during generation: {e}"

# # Example usage (outside a web framework):
# history = []
# user_message = "What is the capital of France?"
# for bot_response in chat(user_message, history):
#     print(f"Bot: {bot_response}")
#     history.append({"role": "user", "content": user_message}) # Important: Add user message to history
#     history.append({"role": "model", "content": bot_response}) # And bot response

# user_message = "What is the weather like there?"
# for bot_response in chat(user_message, history):
#     print(f"Bot: {bot_response}")
#     history.append({"role": "user", "content": user_message})
#     history.append({"role": "model", "content": bot_response})

In [36]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
